In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
from lib.funcs import *

In [2]:
## Get the names and open data
data_domicilio_path = ('./data/CENSO/Domicilio01_PB.csv') 

with open(data_domicilio_path, "r", encoding="latin-1") as myfile:
    for i in range(1):
        line = next(myfile).strip()
        names = line.split(';')


data = pd.read_csv(data_domicilio_path,                  
                sep=';', 
                header=None, 
                names=names,
                skiprows=1,
                encoding="latin-1",
                dtype={'Cod_setor': str})

In [3]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [4]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [5]:
# Select only the data we are interested in
#v001 = Domicílios particulares e domicílios coletivos
domicilios = data[['Cod_setor', 'V001']]
domicilios = domicilios.rename(columns={'V001': 'n_domicilios'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [6]:
# Set the same inde
domicilios.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [7]:
domicilios.to_csv('./data/DOMICILIOS/domicilios_parse.csv')

In [8]:
setores = setores.join(domicilios)

In [9]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [10]:
bairros

,geometry,n_domicilios
NM_BAIRRO,,
Aeroclube,"POLYGON ((-3878845.787 -792159.924, -3878970.3...",3223.0
Altiplano Cabo Branco,"POLYGON ((-3876916.727 -797181.774, -3877032.3...",1466.0
Alto do Céu,"POLYGON ((-3880526.984 -791742.159, -3880526.7...",4651.0
Alto do Mateus,"POLYGON ((-3885968.582 -797638.000, -3886002.0...",4633.0
Anatólia,"POLYGON ((-3878566.488 -797931.549, -3878554.7...",387.0
...,...,...
Trincheiras,"POLYGON ((-3883573.691 -795946.209, -3883573.6...",2061.0
Valentina,"POLYGON ((-3878549.633 -804234.168, -3878556.9...",6318.0
Varadouro,"POLYGON ((-3883870.618 -794980.219, -3883870.6...",1150.0


In [11]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [12]:
bairros_recorte = bairros.loc[bairros.index.isin(bairros_interesse)]
bairros_recorte = bairros_recorte.drop('geometry', axis=1)
bairros_recorte

,n_domicilios
NM_BAIRRO,
Estados,2306.0
Expedicionários,1112.0
Miramar,2814.0
Pedro Gondim,1033.0
Tambauzinho,1520.0
Torre,4734.0
